# Практическая работа №3 по вычислительной математике

Определить первые четыре собственные значения задачи Штурма-Лиувилля

$$
\left\{
\begin{aligned}
\frac{d^2}{dx^2}\left[ (2-x)\frac{d^2y}{dx^2} \right] + \lambda\frac{d^2y}{dx^2} = 0;\\
y(0) = y'_x(0)=y(1)=y''_{xx}(1)=0;
\end{aligned}
\right.\qquad 0 < x < D = 1
$$

предложив численный метод решения задачи. Реализовать его в виде программы, написанной на языке высокого уровня, произвести отладку на модельной задаче и расчёты с относительной погрешностью $\delta = 10^{-4}$.

## Модельная задача

$$
\left\{
\begin{aligned}
&a\frac{d^4y}{dx^4} + \lambda\frac{d^2y}{dx^2} = 0,\qquad a > 0;\\
&y(0) = y'_x(0)=y(1)=y''_{xx}(1)=0;
\end{aligned}
\right.\qquad 0 < x < D = 1
$$

Общее решение уравнения:

$$
y(x) = C_1 \cos\sqrt\frac\lambda a x + C_2 \sin\sqrt\frac\lambda a x + C_3x + C_4
$$

С учётом условий $y(0) = y'_x(0) = 0$ получаем $C_4 = -C_1,\qquad C_3 = -\sqrt{\lambda/a}\ C_2$:

$$
y(x) = C_1 \cos\sqrt\frac\lambda a x + C_2 \sin\sqrt\frac\lambda a x - C_2\sqrt\frac\lambda a x - C_1
$$

Условия $y(1) = y''_{xx}(1) = 0$ дают систему уравнений:
$$
\left\{
\begin{aligned}
&C_1(\cos\sqrt{\lambda/a} - 1) + C_2(\sin\sqrt{\lambda/a} - \sqrt{\lambda/a}) = 0;\\
&(\lambda/a) C_1 \cos\sqrt{\lambda/a} = 0.
\end{aligned}
\right.
$$

Отсюда условия на собственные значения: $\cos\sqrt{\lambda/a} = 0$:

$$
\lambda_n = \frac{a\pi^2(2n+1)^2}{4},\qquad n = 0, 1, 2,\dots
$$

### Объявление и подключение библиотек

In [1]:
%matplotlib inline

import numpy as np
from matplotlib import pylab as plt
from math import log

In [2]:
def sign(x):
    if (x < 0):
        return -1
    elif (x > 0):
        return 1
    return 0

# Создание сетки:
def make_grid(n):
    h = D / (n-1)
    x = np.linspace(start = 0, stop = D, num = n, dtype = float)
    if len(x) != n:
        print('Ошибка с построением сетки, попробуйте выбрать другое n')
        return None
    return x, h

In [3]:
N = 1280
D = 1.

x, h = make_grid(N+1)

In [4]:
def det_type1_sign(k, A, B, C): # k -- размер матрицы
    if k == 1:
        return sign(C[0])
    if A[k-2] == 0.:
        return sign(C[k-1]) * det_type2_sign(k-1, B)
    C[k-2] -= C[k-1]*B[k-2]/A[k-2]
    C[k-1] = 0.
    return -sign(A[k-2]) * det_type1_sign(k-1, A, B, C)
    
def det_type2_sign(k, B):
    ret = 1
    for i in range(k):
        ret *= sign(B[i])
    return ret
    

def det_sign(lbd):
    A = np.zeros(N-1)
    B = np.zeros(N-1)
    A[N-2] = -2*a[N-1]
    B[N-2] = a[N-2] + lbd*h*h
    for n in range(N-2, 0, -1):
        A[n-1] = -(a[n+1]*B[n]+2*A[n]*a[n])
        B[n-1] = A[n]*(a[n-1] + lbd*h*h)
    C = [float(i) for i in range(2*N+1, -1, -2)]
    C[0] = float(N)
    C = np.array(C, dtype = float)
    return det_type1_sign(N, A, B, C)

In [5]:
a = np.full(N+1, 1.)

In [6]:
3.14*3.14/4*9

22.1841

In [8]:
def lbdarray(a, N):
    h = 1./N

    A = np.zeros((N+1, N+1))
    L = np.zeros((N+1, N+1))

    for n in range(0, (N-4)+1):
        A[n][n] = a[n]
        A[n][n+1] = -2*(a[n+1] + a[n])
        A[n][n+2] = a[n+2] + 4*a[n+1] + a[n]
        A[n][n+3] = -2*(a[n+2] + a[n+1])
        A[n][n+4] = a[n+2]

        L[n][n] = h*h
        L[n][n+1] = -2*h*h
        L[n][n+2] = h*h

    L[N-3][0] = 1.
    #L[N-2][0] = -3.
    #L[N-2][1] = 4.
    #L[N-2][2] = -1
    L[N-2][1] = 1.

    L[N-1][N] = 1.

    L[N][N-2] = 1.
    L[N][N-1] = -2.
    L[N][N] = 1.
    B = -np.dot(np.linalg.inv(L), A)
    lbdlist = np.linalg.eigvals(B)
    lbdlist = np.array([z.real for z in lbdlist if z.real > 0 and z.imag == 0], dtype = float)
    lbdlist = np.sort(lbdlist)
    lbdlist = list(lbdlist)
    return lbdlist

In [9]:
N = 1000
a = np.full(N+1, 1.)
ans = lbdarray(a, N)
ans[0:4]

[20.295208832463835, 59.89312163780654, 119.43728265213176, 198.61951872786636]

In [10]:
[(np.pi**2)*((2*n+1)**2) / 4 for n in range(5)]

[2.4674011002723395,
 22.206609902451056,
 61.68502750680849,
 120.90265391334464,
 199.8594891220595]

In [11]:
x = np.linspace(start = 0, stop = 1, num = N+1, dtype = float)
a = 2. - x
ans = lbdarray(a, N)
ans[0:4]

[0.0009038970728799236,
 29.67023053772248,
 87.42545278411704,
 174.31165746686432]